# Projet de programmation

## Introduction <a name="intro"></a>

Il faut récupèrer le projet avec : 

    git clone https://github.com/gwatkinson/projet-python-twitter.git

Puis, il faut créer le fichier `projet/_credentials.py`, qui contient les clés de l'API de Twitter.

Dans le format suivant :

```python
credentials = {
    "consumer_key": "XXXXXXX",
    "consumer_secret": "XXXXXXX",
    "access_token": "XXXXXXX",
    "access_token_secret": "XXXXXXX",
}
```

Finalement, il suffit d'executer les cellules de ce notebook dans l'ordre.

## Table des matières

* [Introduction](#intro)
* [1)Récupération des données](#data)
* [2)Modélisation](#model)
    * [a.Prepocessing](#process)
    * [b.Clustering](#cluster)
* [3)Visualisation](#visu)
    * [a.Nuages de mots](#cloud)
    * [b.Carte interactive](#map)
* [Conclusion](#conc)
* [Annexes](#annex)

## 1) Récupération des données <a name="data"></a>



Nous avons utilisé l'**API** de Twitter pour récupérer les nouveaux tweets publiés sur Twitter, la nuit du 3 au 4 Novembre 2020 (la nuit de l'éléction américaine). Nous avons seulement récupérer les tweets qui contennaient certains mots :

In [ ]:
# Liste 3 sur Trump et Biden uniquement
liste_3 = [
    "biden",
    "trump",
    "JoeBiden",
    "realDonaldTrump",
]

# Liste 4 sur le thème 'vote'
liste_4 = [
    "iwillvote",
    "govote",
    "uselection",
    "vote",
]

# Liste 5 sur le thème 'election'
liste_5 = [
    "uselection",
    "president",
    "presidentialelection",
    "presidential",
    "electionnight",
]

Pour cela, nous avons utilisé le module python `tweepy` ainsi que les fonctions codées dans le module `streaming` (voir la documentation pour plus d'information sur [`start_stream`](https://gwatkinson.github.io/projet-python-twitter/streaming.html#projet_python_twitter.streaming.start_stream)). Voici un exemple d'utilisation du code que nous avons écrit :

In [ ]:
import projet.streaming as stream                           # Contient les fonctions pour le streaming
import projet.listes_mots as listes                         # Contient les listes de mots
import projet._credentials as cred                          # Contient les clés d'authentification à l'API

credentials = stream.CredentialsClass(cred.credentials)     # Pour se connecter à l'API (il faut le fichier projet/_credentials.py)

stream.start_stream(
    credentials=credentials,
    liste_mots=listes.liste_3,                              # Liste des mots à tracker (voir `projet.listes_mots`)
    nb=200,                                                 # Nombre de tweets à recupérer
    # timeout=10/3600,                                        # Durée du stream
    fprefix="exemple_liste_3",                              # À modifier en fonction de la liste selectionnée
    path="./data/json/",                                    # À modifier selon l'utilisateur (doit finir par "/" ou "\")
    verbose=True,
)

Un fichier `json` a été créé dans `data/json/`.

Pour voir à quoi ressemble les données :

In [ ]:
import glob
import json
import pandas as pd

path = glob.glob("data/json/exemple_liste_3*.json")[-1]  # On récupère le dernier fichier exemple crée
print("On regarde le fichier : "+path+"\n")

tweets_list = []
with open(path, "r") as fh:
    file = fh.read().split("\n")
    for line in file:
        if line:
            tweets_list.append(json.loads(line))

print("Le premier tweet :")
print(tweets_list[0])


Il s'agit du format `json`. Il est difficile de voir les variable comme cela. On peut créer une `dataframe pandas` pour mieux comprendre les données.

In [ ]:
df_tweets = pd.DataFrame(tweets_list)
df_tweets.head()

In [ ]:
print("Dimensions : ", df_tweets.shape)

In [ ]:
print("Colonnes :\n")
for name in list(df_tweets):
    print(name)

## 2) Modélisation <a name="model"></a>

### a. Preprocessing <a name="process"></a>

Nous avons fait une fonction qui fait les étapes précédentes ainsi que des fonctions pour nettoyer les données. Elles sont dans le fichier `processing.py`.

In [ ]:
import projet.processing as process                         # Contient les fonctions pour le processing de la dataframe

folder = "./data/json/"                                     # Pour écupèrer tous les fichiers json dans le dossier 'data/json/'

dirty_df = process.tweet_json_to_df(folder=folder, verbose=True)     # Convertit les json en dataframe pandas


Nous avons ainsi récupérer les fichiers dans `data/json/` dans la dataframe pandas `dirty_df`.

Elle ressemble à :

In [ ]:

dirty_df.head()

On peut ensuite utiliser `clean_df` pour filtrer et nettoyer la base de donnée en conservant seulement les informations qui nous interressent. On peut aussi utiliser une liste de `listes_variables` pour récupérer d'autres variables ou en ajouter dans l'option `extra` de `clean_df`. Voir la doc pour plus de détails : [`clean_df`](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.clean_df).

In [ ]:
clean_df = process.clean_df(dirty_df, index="id", date="created_at", verbose=True)

In [ ]:
text_df = process.get_full_text(clean_df, new_var="full_text", drop_vars=True)

In [ ]:
from importlib import reload
reload(process)

In [ ]:
politic_df = process.add_politics(text_df)

In [ ]:
politic_df[["full_text", "full_text-contains_trump", "full_text-contains_biden"]].head()

In [ ]:
from importlib import reload
reload(process)

In [ ]:
sentiment_df = process.add_sentiment(politic_df)

In [ ]:
sentiment_df["full_textAjout-sentiment-compound"].plot.hist()

In [ ]:
final_df = process.sentiment_class(sentiment_df)

In [ ]:
sum(final_df["user-description-contains_trump"].isnull())/len(final_df)

In [ ]:
sum(~final_df["user-location"].isnull())/len(final_df)

In [ ]:
import projet.modelisation as model

In [ ]:
std_mat = model.standardize(final_df, model.get_numeric(final_df.drop("user-id", axis=1)))

In [ ]:
std_mat

In [ ]:
from importlib import reload
reload(model)

In [ ]:
kmeans, sse = model.add_group(std_mat, max_cluster=30)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.style.use("fivethirtyeight")
plt.plot(range(1, 31), sse)
plt.xticks(range(1, 31))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
kmeans.cluster_centers_

### b. Clustering <a name="cluster"></a>

## 3) Visualisation <a name="visu"></a>

### a. Nuages de mots <a name="cloud"></a>

### b. Carte interactive <a name="map"></a>

## Conclusion <a name="conc"></a>

## Annexes <a name="annex"></a>